In [0]:
df_rome        = spark.read.format("delta").table("silver.rome")
df_london      = spark.read.format("delta").table("silver.london")
df_amsterdam   = spark.read.format("delta").table("silver.amsterdam")
df_athens      = spark.read.format("delta").table("silver.athens")
df_barcelona   = spark.read.format("delta").table("silver.barcelona")
df_berlin      = spark.read.format("delta").table("silver.berlin")
df_lisbon      = spark.read.format("delta").table("silver.lisbon")
df_paris       = spark.read.format("delta").table("silver.paris")


In [0]:
from pyspark.sql import functions as F

def arredondar_dist_tamanho_indices(df):
    """
    Arredonda colunas de:
    - distâncias e metros
    - capacidade e quartos
    - índices normalizados
    - preço total
    """
    for c in df.columns:
        lower_c = c.lower()
        if any(k in lower_c for k in ["dist", "metro", "capacit", "bedrooms", "normalizado"]):
            df = df.withColumn(c, F.round(F.col(c), 4))
        elif "preco_total_diario" == lower_c:
            df = df.withColumn(c, F.round(F.col(c), 2))
    return df


def converter_para_booleano(df, colunas):
    """Converte colunas 0/1 para booleano"""
    for c in colunas:
        if c in df.columns:
            df = df.withColumn(c, F.col(c).cast("boolean"))
    return df

def remover_colunas(df):
    """Remove colunas de quarto e auditoria."""
    colunas_para_remover = [
        "QUARTO_PRIVADO", "QUARTO_COMPARTILHADO",
        "DATA_ARQUIVO_SILVER", "NOME_ARQUIVO_BRONZE",
        "DATA_HORA_BRONZE", "NOME_ARQUIVO",
        "INDICE_ATRACOES", "INDICE_RESTAURANTES"
    ]
    existing = [c for c in colunas_para_remover if c in df.columns]
    return df.drop(*existing)

def gerar_gold(df, cidade):
    """
    Prepara DataFrame Gold:
    - Remove colunas de quarto e auditoria
    - Converte para booleano
    - Arredonda distâncias, tamanhos, índices normalizados e preço
    - Cria coluna de avaliação média
    - Adiciona coluna CIDADE
    """
    df = remover_colunas(df)
    df = converter_para_booleano(df, ["IMOVEIS_ANFITRIAO", "HOSPEDAGEM_CORPORATIVA"])
    df = arredondar_dist_tamanho_indices(df)

    if "NOTA_LIMPEZA" in df.columns and "NOTA_SATISFACAO_HOSPEDES" in df.columns:
        df = df.withColumn(
            "AVALIACAO_MEDIA",
            F.round((F.col("NOTA_LIMPEZA") + F.col("NOTA_SATISFACAO_HOSPEDES")) / 2, 2)
        )
        df = df.drop("NOTA_LIMPEZA", "NOTA_SATISFACAO_HOSPEDES")

    df_gold = df.withColumn("CIDADE", F.lit(cidade))
    return df_gold


In [0]:
cidades_dfs = {
    "rome": df_rome,
    "london": df_london,
    "amsterdam": df_amsterdam,
    "athens": df_athens,
    "barcelona": df_barcelona,
    "berlin": df_berlin,
    "lisbon": df_lisbon,
    "paris": df_paris
}


In [0]:
for cidade, df in cidades_dfs.items():
    df_gold = gerar_gold(df, cidade)
    
    # Aqui convertemos colunas específicas para boolean antes de salvar
    df_gold = converter_para_booleano(df_gold, ["IMOVEIS_ANFITRIAO", "HOSPEDAGEM_CORPORATIVA"])
    
    df_gold.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(f"gold.{cidade}")

